# 02 Limpieza de Datos

Objetivo del Notebook: El objetivo principal de este notebook es identificar las columnas en la base de datos que corresponden a las variables de la fórmula de Kothari para el cálculo de devengos discrecionales.

In [13]:
# Cargar la sesión de Kedro y el dataset de la hoja 2
%load_ext kedro.ipython
%reload_kedro

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


[11/01/25 18:37:44] INFO     Resolved project path as:                                              ]8;id=289428;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=743055;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#180\180]8;;\
                             /home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/k                
                             otari-gender-data-analysis.                                                           
                             To set a different path, run '%reload_kedro <project_root>'                           

                    INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=577745;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=706693;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro_telemetry/plugin.py#243\243]8;;\
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[11/01/25 18:37:45] INFO     Kedro project kotari-gender-data-analysis                              ]8;id=528522;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=498671;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#146\146]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=951357;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=277255;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#147\147]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=784938;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=254968;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#153\153]8;;\

El modelo se basa en el cálculo de los devengos totales y luego estima los devengos no discrecionales (los normales o esperados) usando regresiones. 

$$
\frac{TA_{i,t}}{A_{i,t-1}} = \alpha_0 + \alpha_1(\frac{1}{A_{i,t-1}}) + \alpha_2(\frac{(\Delta REV_{i,t} - \Delta REC_{i,t})}{A_{i,t-1}}) + \alpha_3(\frac{PPE_{i,t}}{A_{i,t-1}}) + \alpha_4 ROA_{i,t} + \varepsilon_{i,t}
$$

Donde:

- $TA_{i,t}$: Devengos totales de la empresa $i$ en el año $t$ (diferencia entre utilidad neta y flujo de caja operativo.)
- $A_{i,t-1}$: Activos totales al inicio del año $t$
- $\Delta REV_{i,t}$ (Revenue): Cambio en ingresos (ventas) entre $t$ y $t-1$
- $\Delta REC_{i,t}$: Cambio en cuentas por cobrar entre $t$ y $t-1$
- $PPE_{i,t}$: Propiedades, planta y equipo
- $ROA_{i,t}$: Retorno sobre activos
- $\varepsilon_{i,t}$: devengo discrecional (Componente no explicado)

Una vez identificadas estas variables, se creará un nuevo DataFrame que solo contenga las filas donde todas estas variables estén presentes (sin valores faltantes), para realizar el análisis exploratorio y posteriores cálculos del modelo.

## Identificacion de columnas de la base de datos

| Columna | Nombre en la formula |
|---------|----------------|
| EMPRESA | id |
| ANIO | t |
| UTILIDAD NETA | $TA_{i,t}$ es $UN - CFO$ |
| ROA | $ROA_{i,t}$ |
| Cantidad de mujeres en Directorios (incluye suplentes) | N_muj_dir |
| Total en Directorios | N_dir |
| Cantidad de mujeres en otros ejecutivos | N_muj_ejec |
| Total en otros ejecutivos | N_ejec |
| Flujo efectivo de operación (CFO) | $TA_{i,t}$ es $UN - CFO$ |
| Activos total al inicio de anio | $A_{i,t-1}$ |
| Ventas del anio (diferencia) | $\Delta REV_{i,t}$ |
| Cuentas por cobrar (diferencia) | $\Delta REC_{i,t}$ |
| Propiedades, planta y equipo (PPE) | $PPE_{i,t}$ |


## Filtrar por las columnas anteriores las que tienen NAN

### Cargar data

In [14]:
data = catalog.load("raw_mineria_data")
sheet_name = "BASE DE DATOS"
df = data[sheet_name]
print(f"Se analizará la hoja: {sheet_name}")
print(f"Dimensiones: {df.shape[0]} filas × {df.shape[1]} columnas")

                    INFO     Loading data from raw_mineria_data (ExcelDataset)...               ]8;id=902873;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=600026;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#403\403]8;;\

Se analizará la hoja: BASE DE DATOS
Dimensiones: 209 filas × 24 columnas


In [15]:
# Mapeo manual de las variables de la fórmula a las columnas del DataFrame
# (Ajusta los nombres según los nombres reales en tu base de datos)
kothari_vars = {
    'Id': 'EMPRESA',
    't': 'ANIO',
    'UN': 'UTILIDAD NETA',
    'ROA': 'ROA',
    'N_muj_dir': 'Cantidad de mujeres en Directorios (incluye suplentes)',
    'N_dir': 'Total en Directorios',
    'N_muj_ejec': 'Cantidad de mujeres en otros ejecutivos',
    'N_ejec': 'Total en otros ejecutivos',
    'CFO': 'Flujo efectivo de operación (CFO)',
    'A_it_1': 'Activos total al inicio de anio',
    'Delta_REV_it': 'Ventas del anio (diferencia)',
    'Delta_REC_it': 'Cuentas por cobrar (diferencia)',
    'PPE_it': 'Propiedades, planta y equipo (PPE)' 
}

# Verificar si todas las columnas existen en el DataFrame
def check_columns(mapping, dataframe):
    for var, col in mapping.items():
        if col not in dataframe.columns:
            print(f"⚠️ La columna '{col}' para la variable '{var}' no existe en el DataFrame.")
        else:
            print(f"✓ Variable '{var}' mapeada a columna '{col}'.")

check_columns(kothari_vars, df)

✓ Variable 'Id' mapeada a columna 'EMPRESA'.
✓ Variable 't' mapeada a columna 'ANIO'.
✓ Variable 'UN' mapeada a columna 'UTILIDAD NETA'.
✓ Variable 'ROA' mapeada a columna 'ROA'.
✓ Variable 'N_muj_dir' mapeada a columna 'Cantidad de mujeres en Directorios (incluye suplentes)'.
✓ Variable 'N_dir' mapeada a columna 'Total en Directorios'.
✓ Variable 'N_muj_ejec' mapeada a columna 'Cantidad de mujeres en otros ejecutivos'.
✓ Variable 'N_ejec' mapeada a columna 'Total en otros ejecutivos'.
✓ Variable 'CFO' mapeada a columna 'Flujo efectivo de operación (CFO)'.
✓ Variable 'A_it_1' mapeada a columna 'Activos total al inicio de anio'.
✓ Variable 'Delta_REV_it' mapeada a columna 'Ventas del anio (diferencia)'.
✓ Variable 'Delta_REC_it' mapeada a columna 'Cuentas por cobrar (diferencia)'.
✓ Variable 'PPE_it' mapeada a columna 'Propiedades, planta y equipo (PPE)'.


In [16]:
# Filtrar el DataFrame para conservar solo las filas sin NaNs en las columnas relevantes
df_filtrado = df.dropna(subset=[col for col in kothari_vars.values() if col in df.columns])
# Eliminar filas donde 'Activos total al inicio de anio' sea igual a 0
col_activos = 'Activos total al inicio de anio'
if col_activos in df_filtrado.columns:
    df_kothari = df_filtrado[df_filtrado[col_activos] != 0]
else:
    df_kothari = df_filtrado.copy()

print(f"\nFilas originales: {df.shape[0]}")
print(f"Filas después de eliminar NaNs y ceros en '{col_activos}': {df_kothari.shape[0]}")
print(f"Columnas utilizadas para el modelo: {[col for col in kothari_vars.values() if col in df.columns]}")

# Mostrar cuántas filas se eliminaron y de qué empresas
eliminadas = df.loc[~df.index.isin(df_kothari.index)]
print(f"Filas eliminadas por NaN o ceros: {eliminadas.shape[0]}")
if 'EMPRESA' in eliminadas.columns:
    print("Empresas afectadas:")
    print(eliminadas['EMPRESA'].unique())
else:
    print("No se encontró la columna 'EMPRESA' en las filas eliminadas.")

# Mostrar cuántas empresas únicas había y cuántas quedan
if 'EMPRESA' in df.columns and 'EMPRESA' in df_kothari.columns:
    empresas_originales = df['EMPRESA'].nunique()
    empresas_filtradas = df_kothari['EMPRESA'].nunique()
    print(f"Empresas únicas originales: {empresas_originales}")
    print(f"Empresas únicas después del filtrado: {empresas_filtradas}")
else:
    print("No se encontró la columna 'EMPRESA' para contar empresas únicas.")

# Mostrar una muestra del nuevo DataFrame filtrado
display(df_kothari.head())

# Mostrar la cantidad de filas que quedaron por empresa
grupo_empresas = df_kothari['EMPRESA'].value_counts()
print("Cantidad de filas por empresa después del filtrado:")
print(grupo_empresas)


Filas originales: 209
Filas después de eliminar NaNs y ceros en 'Activos total al inicio de anio': 158
Columnas utilizadas para el modelo: ['EMPRESA', 'ANIO', 'UTILIDAD NETA', 'ROA', 'Cantidad de mujeres en Directorios (incluye suplentes)', 'Total en Directorios', 'Cantidad de mujeres en otros ejecutivos', 'Total en otros ejecutivos', 'Flujo efectivo de operación (CFO)', 'Activos total al inicio de anio', 'Ventas del anio (diferencia)', 'Cuentas por cobrar (diferencia)', 'Propiedades, planta y equipo (PPE)']
Filas eliminadas por NaN o ceros: 51
Empresas afectadas:
['ANGLO AMERICAN SUR S.A. (EX MINERA SUR ANDES LTDA.)'
 'COMPAÑIA MINERA MANTOS DE ORO' 'COMPAÑIA MINERA MARICUNGA S.A.'
 'COMPAÑIA MINERA NEVADA SPA' 'COMPAÑIA CONTRACTUAL MINERA CANDELARIA'
 'COMPAÑIA MINERA CERRO COLORADO LTDA.'
 'COMPAÑIA MINERA DOÑA INÉS DE COLLAHUASI SCM'
 'COMPAÑIA MINERA TECK CARMEN DE ANDACOLLO'
 'COMPAÑIA MINERA TECK QUEBRADA BLANCA S.A.' 'MINERA ANTUCOYA'
 'MINERA CENTINELA' 'MINERA ESCONDIDA LIMI

,RUT,EMPRESA,COTIZA EN BOLSA,EMP. FAMI,ANIO,UTILIDAD NETA,TOTAL ACTIVOS,PATRIMONIO,UTILIDAD OPERACIONAL,DEPRECIACIÓN Y AMORTIZACIÓN,...,Total en Directorios,Cantidad de mujeres en otros ejecutivos,Total en otros ejecutivos,Flujo efectivo de operación (CFO),Activos total al inicio de anio,Ventas,Ventas del anio (diferencia),Cuentas por cobrar,Cuentas por cobrar (diferencia),"Propiedades, planta y equipo (PPE)"
0,76000957-1,ALGORTA NORTE S.A.,NO,NO,2013,16634000.0,256528000.0,105248000.0,26480000.0,17691000.0,...,8.0,1,10,1522000.0,239444000.0,90936000.0,36675000.0,24144000.0,12412000.0,175848000.0
1,76000957-1,ALGORTA NORTE S.A.,NO,NO,2014,20958000.0,236873000.0,97937000.0,33104000.0,17590000.0,...,8.0,1,10,27073000.0,256528000.0,101852000.0,10916000.0,40015000.0,15871000.0,157938000.0
2,76000957-1,ALGORTA NORTE S.A.,NO,NO,2015,13382000.0,229034000.0,94895000.0,27749000.0,18313000.0,...,8.0,1,10,4457000.0,236873000.0,83480000.0,-18372000.0,31112000.0,-8903000.0,136090000.0
3,76000957-1,ALGORTA NORTE S.A.,NO,NO,2016,-8914000.0,212738000.0,93755000.0,-7622000.0,15377000.0,...,8.0,1,10,9530000.0,229034000.0,66295000.0,-17185000.0,31274000.0,162000.0,124893000.0
4,76000957-1,ALGORTA NORTE S.A.,NO,NO,2017,-11637000.0,195704000.0,82053000.0,-12280000.0,17285000.0,...,8.0,1,9,7830000.0,212738000.0,62031000.0,-4264000.0,21727000.0,-9547000.0,109692000.0


Cantidad de filas por empresa después del filtrado:
EMPRESA
ALGORTA NORTE S.A.                                     11
MINERA SPENCE S.A.                                     11
NORTE ABIERTO SPA                                      11
SOCIEDAD CONTRACTUAL MINERA EL ABRA                    11
SOCIEDAD CONTRACTUAL MINERA EL MORRO                   11
COMPAÑIA MINERA MANTOS DE ORO                          10
COMPAÑIA MINERA NEVADA SPA                             10
COMPAÑIA MINERA CERRO COLORADO LTDA.                   10
COMPAÑIA MINERA TECK CARMEN DE ANDACOLLO               10
MINERA ESCONDIDA LIMITADA                              10
MINERA LOS PELAMBRES                                   10
COMPAÑIA MINERA MARICUNGA S.A.                          9
COMPAÑIA MINERA DOÑA INÉS DE COLLAHUASI SCM             9
MINERA ANTUCOYA                                         9
CODELCO                                                 9
ANGLO AMERICAN SUR S.A. (EX MINERA SUR ANDES LTDA.)     7
Name: count,

### Revisar si 'Activos total al inicio de anio' no es cero

In [17]:
# Revisar si 'Activos total al inicio de anio' tiene valores cero
col_name = 'Activos total al inicio de anio'
if col_name in df_kothari.columns:
    zero_count = (df_kothari[col_name] == 0).sum()
    print(f"Filas con '{col_name}' igual a cero: {zero_count}")
    if zero_count > 0:
        print(df_kothari[df_kothari[col_name] == 0])
else:
    print(f"La columna '{col_name}' no existe en el DataFrame filtrado.")


Filas con 'Activos total al inicio de anio' igual a cero: 0


### Revisar si 'TOTAL ACTIVOS' mp tiene ceros.

In [18]:
# Revisar si 'TOTAL ACTIVOS' tiene valores cero
col_name = 'TOTAL ACTIVOS'
if col_name in df_kothari.columns:
    zero_count = (df_kothari[col_name] == 0).sum()
    print(f"Filas con '{col_name}' igual a cero: {zero_count}")
    if zero_count > 0:
        print(df_kothari[df_kothari[col_name] == 0])
else:
    print(f"La columna '{col_name}' no existe en el DataFrame filtrado.")

Filas con 'TOTAL ACTIVOS' igual a cero: 0


## Crear variable de peso de la empresa

In [19]:
# Calcular el peso relativo de cada empresa según activos totales
peso_empresas = df_kothari.groupby('EMPRESA')['TOTAL ACTIVOS'].sum().reset_index()
peso_empresas['peso_empresa'] = (peso_empresas['TOTAL ACTIVOS'] / peso_empresas['TOTAL ACTIVOS'].sum()) * 100

# Calcular el peso de mujeres en directorio por empresa
peso_muj_dir = df_kothari.groupby('EMPRESA')[['Cantidad de mujeres en Directorios (incluye suplentes)', 'Total en Directorios']].sum().reset_index()
peso_muj_dir['peso_muj_dir'] = (peso_muj_dir['Cantidad de mujeres en Directorios (incluye suplentes)'] / peso_muj_dir['Total en Directorios']) * 100

# Calcular el peso de mujeres ejecutivas por empresa
peso_muj_ejec = df_kothari.groupby('EMPRESA')[['Cantidad de mujeres en otros ejecutivos', 'Total en otros ejecutivos']].sum().reset_index()
peso_muj_ejec['peso_muj_ejec'] = (peso_muj_ejec['Cantidad de mujeres en otros ejecutivos'] / peso_muj_ejec['Total en otros ejecutivos']) * 100

# Unir los pesos al DataFrame principal
df_kothari = df_kothari.merge(peso_empresas[['EMPRESA', 'peso_empresa']], on='EMPRESA', how='left')
df_kothari = df_kothari.merge(peso_muj_dir[['EMPRESA', 'peso_muj_dir']], on='EMPRESA', how='left')
df_kothari = df_kothari.merge(peso_muj_ejec[['EMPRESA', 'peso_muj_ejec']], on='EMPRESA', how='left')

## Crear variable de devengos totales

In [20]:
df_kothari['TA'] = df_kothari['UTILIDAD NETA'] - df_kothari['Flujo efectivo de operación (CFO)']

## Guardar data filtrada


In [21]:
# Guardar el DataFrame filtrado en el catálogo de Kedro como Parquet
catalog.save("filtered_kothari_data", df_kothari)
print("DataFrame filtrado guardado en 'filtered_kothari_data' como Parquet.")

                    INFO     Saving data to filtered_kothari_data (ParquetDataset)...           ]8;id=198202;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=899909;file:///home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#445\445]8;;\

DataFrame filtrado guardado en 'filtered_kothari_data' como Parquet.


## Estadisticas de la data filtrada

In [22]:
# Número de filas por empresa en el DataFrame filtrado
empresas_filas = df_kothari['EMPRESA'].value_counts()
print("Número de filas por empresa:")
print(empresas_filas)

Número de filas por empresa:
EMPRESA
ALGORTA NORTE S.A.                                     11
MINERA SPENCE S.A.                                     11
NORTE ABIERTO SPA                                      11
SOCIEDAD CONTRACTUAL MINERA EL ABRA                    11
SOCIEDAD CONTRACTUAL MINERA EL MORRO                   11
COMPAÑIA MINERA MANTOS DE ORO                          10
COMPAÑIA MINERA NEVADA SPA                             10
COMPAÑIA MINERA CERRO COLORADO LTDA.                   10
COMPAÑIA MINERA TECK CARMEN DE ANDACOLLO               10
MINERA ESCONDIDA LIMITADA                              10
MINERA LOS PELAMBRES                                   10
COMPAÑIA MINERA MARICUNGA S.A.                          9
COMPAÑIA MINERA DOÑA INÉS DE COLLAHUASI SCM             9
MINERA ANTUCOYA                                         9
CODELCO                                                 9
ANGLO AMERICAN SUR S.A. (EX MINERA SUR ANDES LTDA.)     7
Name: count, dtype: int64


In [23]:
# Estadísticos descriptivos para variables relevantes de Kothari (incluyendo TA, excluyendo EMPRESA y ANIO)
import scipy.stats as stats
exclude_vars = ['EMPRESA', 'ANIO']
relevant_vars = [col for col in kothari_vars.values() if col in df_kothari.columns and col not in exclude_vars]
relevant_vars.extend(['TA', 'peso_empresa', 'peso_muj_dir', 'peso_muj_ejec'])

summary_stats = []
for var in relevant_vars:
    data_var = df_kothari[var].dropna()
    N = data_var.count()
    min_ = data_var.min()
    max_ = data_var.max()
    mean_ = data_var.mean()
    std_ = data_var.std()
    skew_ = stats.skew(data_var) if N > 2 else float('nan')
    kurt_ = stats.kurtosis(data_var) if N > 2 else float('nan')
    summary_stats.append([var, N, min_, max_, mean_, std_, skew_, kurt_])
import pandas as pd
summary_df = pd.DataFrame(summary_stats, columns=["Variable", "N", "Minimum", "Maximum", "Mean", "Std. Dev", "Skewness", "Kurtosis"])
display(summary_df)

,Variable,N,Minimum,Maximum,Mean,Std. Dev,Skewness,Kurtosis
0,UTILIDAD NETA,158,-2.191000e+09,3.889000e+09,3.519225e+08,8.303679e+08,1.846983,4.901949
1,ROA,158,-1.149301e+02,4.766844e+01,9.055123e-02,2.169744e+01,-2.800061,10.928386
2,Cantidad de mujeres en Directorios (incluye su...,158,0.000000e+00,6.000000e+00,9.683544e-01,1.164337e+00,1.154543,1.170715
3,Total en Directorios,158,0.000000e+00,1.400000e+01,6.177215e+00,4.586770e+00,-0.029064,-0.997530
4,Cantidad de mujeres en otros ejecutivos,158,0.000000e+00,6.000000e+00,6.645570e-01,1.137719e+00,2.245228,5.202914
5,Total en otros ejecutivos,158,0.000000e+00,3.300000e+01,7.651899e+00,5.573970e+00,1.130790,2.140170
6,Flujo efectivo de operación (CFO),158,-1.734580e+08,8.301000e+09,8.001927e+08,1.460382e+09,2.473686,6.459922
7,Activos total al inicio de anio,158,1.289540e+08,4.082400e+10,4.729168e+09,8.521279e+09,2.891762,7.880154
8,Ventas del anio (diferencia),158,-2.797000e+09,5.231138e+09,4.060451e+07,6.800725e+08,2.517458,22.803501
9,Cuentas por cobrar (diferencia),158,-6.031430e+08,9.450330e+08,9.134136e+06,1.657610e+08,1.515991,10.288849


In [24]:
# Importar librerías necesarias para el análisis de la hoja "LISTADO EMPRESA"
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

## Exportar resultados a markdown

In [25]:
# Exportar estadísticos descriptivos de variables Kothari a markdown
output_path = "/home/gabriel/Escritorio/Proyectos/UBO/Erik_Escalona/kotari_proyecto/kotari-gender-data-analysis/docs/source/note_02_Kothari_Stats.md"

with open(output_path, "w", encoding="utf-8") as f:
    f.write("# Estadísticos Descriptivos de Variables Relevantes Kothari\n\n")
    f.write("<!-- Este archivo muestra los estadísticos descriptivos de las variables relevantes para el modelo de Kothari. -->\n\n")
    f.write("**Fuente:** notebooks/02_Clean_Data.ipynb\n\n")
    f.write("## Resumen de limpieza de datos\n\n")
    f.write("En este notebook se realizó la limpieza de la base de datos para el modelo de Kothari. Se identificaron las columnas relevantes, se eliminaron las filas con valores faltantes (NaN) en dichas columnas y aquellas donde los activos totales al inicio del año eran cero. Además, se crearon variables adicionales como el peso relativo de cada empresa y los devengos totales. El resultado es un DataFrame filtrado y listo para el análisis exploratorio y el cálculo de los devengos discrecionales.\n\n")
    f.write("## Tabla de Estadísticos\n\n")
    f.write("| Variable | N | Minimum | Maximum | Mean | Std. Dev | Skewness | Kurtosis |\n")
    f.write("|----------|---|---------|---------|------|----------|----------|----------|\n")
    for _, row in summary_df.iterrows():
        f.write(f"| {row['Variable']} | {row['N']} | {row['Minimum']:.2f} | {row['Maximum']:.2f} | {row['Mean']:.2f} | {row['Std. Dev']:.2f} | {row['Skewness']:.2f} | {row['Kurtosis']:.2f} |\n")
    f.write("\n*Generado automáticamente desde el notebook.*\n")